## Callin Switzer
April 2019

Multiprocessing with multiple arguments

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
import seaborn as sns
from scipy.integrate import odeint
import random
import time
from datetime import datetime
import sys
from multiprocessing import Pool, cpu_count
import simUtils # note that this is a custom-written file 

print(sys.version)

3.6.7 (default, Feb 28 2019, 07:28:18) [MSC v.1900 64 bit (AMD64)]


In [3]:
now = datetime.now()
print("last run on " + str(now))


last run on 2019-04-26 15:13:48.509869


In [4]:
K = 29.3 

# c is the torsional damping constant of the thorax-petiole joint
# in (cm^2)*g/s
c =  14075.8   

# refref: change. Gravity set to 0 for testing
g =  980.0   #g is the acceleration due to gravity in cm/(s^2)
#g =  0.0   #g is the acceleration due to gravity in cm/(s^2)

# This is the resting configuration of our 
# torsional spring(s) = Initial abdomen angle - initial head angle - pi
betaR =  0.0 

In [17]:
globalDict = {"bhead": 0.507,
            "ahead": 0.908,
            "bbutt": 0.1295,
            "abutt": 1.7475, 
            "rho": 1, 
            "rhoA": 0.00118, 
            "muA": 0.000186, 
            "L1": 0.908, 
            "L2": 1.7475,  
            "L3": 0.75,
            "K": 29.3,
            "c":  14075.8,
            "g": 980.0,
            "betaR":  0.0,
            }

In [24]:
# refref: here

globalDict["m2"] = globalDict["rho"]*(4/3)*np.pi*(globalDict["bbutt"]**2)*globalDict["abutt"]
globalDict["echead"] = globalDict["ahead"]/globalDict["bhead"]
globalDict['ecbutt'] = globalDict['abutt']/globalDict['bbutt']
globalDict

{'bhead': 0.507,
 'ahead': 0.908,
 'bbutt': 0.1295,
 'abutt': 1.7475,
 'rho': 1,
 'rhoA': 0.00118,
 'muA': 0.000186,
 'L1': 0.908,
 'L2': 1.7475,
 'L3': 0.75,
 'K': 29.3,
 'c': 14075.8,
 'g': 980.0,
 'betaR': 0.0,
 'm2': 0.12275673548335365,
 'echead': 1.7909270216962525,
 'ecbutt': 13.494208494208495}

In [ ]:
# calculated variables for data dict
globalDict["m2"] = globalDict["rho"]*(4/3)*np.pi*(globalDict["bbutt"]**2)*globalDict["abutt"]

echead = ahead/bhead; #Eccentricity of head-thorax (unitless)
ecbutt = abutt/bbutt; #Eccentricity of gaster (unitless)
I1 = (1/5)*m1*(bhead**2)*(1 + echead**2); #Moment of inertia of the 
    #head-thorax
I2 = (1/5)*m2*(bbutt**2)*(1 + ecbutt**2); #Moment of inertia of the gaster

#This is the surface area of the object experiencing drag.
S_head = np.pi*bhead**2

#This is the surface area of the object experiencing drag.
S_butt = np.pi*bbutt**2 

In [18]:
dataDict = {"globalVars": globalDict}
dataDict

{'globalVars': {'bhead': 0.507,
  'ahead': 0.908,
  'bbutt': 0.1295,
  'abutt': 1.7475,
  'rho': 1,
  'rhoA': 0.00118,
  'muA': 0.000186,
  'L1': 0.908,
  'L2': 1.7475,
  'L3': 0.75,
  'K': 29.3,
  'c': 14075.8,
  'g': 980.0,
  'betaR': 0.0}}

In [5]:

# parallel run of simulations -- this generates approximately 1 million simulations per hour
p = Pool(cpu_count()-1)
stt = time.time()   
# bb = p.map(multiProcTraj.flyBug_firstLast, range(100000) ) 
bb = p.map(simUtils.flyBug_firstLast, range(5) ) 
print(time.time() -stt)

p.close()
p.join()
print(time.time() -stt)

2.3627583980560303
2.632336139678955


In [6]:
len(bb)

5

In [9]:
# save data to csv
newDF = pd.DataFrame(bb, columns= ["x0", "xf", "xd0", "xdf", "y0", "yf", "yd0", "ydf", "theta0", \
"thetaf", "thetad0", "thetadf", "phi0", "phif", "phid0", "phidf", "F", "alpha", "tau0"])


newDF.head()

,x0,xf,xd0,xdf,y0,yf,yd0,ydf,theta0,thetaf,thetad0,thetadf,phi0,phif,phid0,phidf,F,alpha,tau0
0,0.0,24.340899,1313.773246,1118.739278,0.0,25.365155,1279.593660,1261.945105,3.877824,3.809095,-14.627348,12.184288,3.486631,3.503013,-15.963679,16.447035,9525.924585,5.710089,-59883.556340
1,0.0,-6.030707,-209.478331,-404.119399,0.0,-18.573711,-1030.792267,-848.883591,3.550411,4.016087,14.486710,22.036513,1.807386,2.155328,23.072203,16.139997,14952.149048,4.823505,82087.803871
2,0.0,16.672989,653.763181,636.468054,0.0,-13.893948,-883.714150,-704.563706,5.288458,1.684408,-2.421259,-371.482559,5.487810,1.817826,20.010103,-374.645602,44206.927211,2.566020,46751.769057
3,0.0,-20.031843,-879.445123,-1402.360768,0.0,27.280875,1025.418672,1526.083675,1.639914,3.624595,-3.114772,192.303518,2.543247,4.624861,6.238099,197.428560,37678.572352,0.107532,-63814.407173
4,0.0,11.464058,524.726063,708.510661,0.0,21.311090,745.852501,1356.139627,3.555651,2.478600,11.553184,-102.581083,5.607401,4.662310,-15.913364,-95.909443,43072.386005,4.381097,-96592.023858


In [7]:
# save data to csv
newDF = pd.DataFrame(bb, columns= ["x0", "xf", "xd0", "xdf", "y0", "yf", "yd0", "ydf", "theta0", \
"thetaf", "thetad0", "thetadf", "phi0", "phif", "phid0", "phidf", "F", "alpha", "tau0"])


newDF.head()

,x0,xf,xd0,xdf,y0,yf,yd0,ydf,theta0,thetaf,thetad0,thetadf,phi0,phif,phid0,phidf,F,alpha,tau0
0,0.0,-15.583832,-798.135971,-776.795062,0.0,4.241895,117.571616,322.114175,0.565720,0.728147,-21.598231,18.386510,3.927618,4.208466,18.478888,24.290336,13794.749225,0.955305,-82154.467965
1,0.0,20.694136,760.978307,1206.209327,0.0,5.857230,500.507780,36.114042,1.844925,0.849353,-20.252023,-103.758826,4.685064,3.641893,19.863159,-106.123112,39423.110830,3.966777,29990.238672
2,0.0,-18.286874,-735.627592,-891.439751,0.0,21.909691,950.266900,1349.169432,1.750889,0.049342,15.708582,-166.638607,1.898136,0.122352,-14.938776,-170.318759,18381.485588,0.875628,51661.267162
3,0.0,-16.407055,-880.277711,-762.137338,0.0,3.908073,240.148055,143.357324,5.692428,5.380496,2.188986,-35.352477,4.171733,3.842600,4.913136,-36.225159,6903.613230,6.156277,11651.262057
4,0.0,-1.305426,-118.857385,-10.712816,0.0,-9.122142,-465.474879,-469.147602,3.793257,3.268752,-22.062717,-43.507957,5.319005,4.807174,1.601871,-42.875572,6202.305362,3.010038,-9042.907675


In [7]:
tstamp = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
print(tstamp)

2019_04_03__10_00_31


In [8]:
pythonMadeData

'D:/Dropbox/AcademiaDropbox/mothMachineLearning_dataAndFigs/PythonGeneratedData/TestData'

In [9]:
newDF.to_csv(os.path.join(pythonMadeData, 
                          "RandomInitCond_" + tstamp + ".csv"),
             index = False)

In [18]:
# for loop to run parallel processes a few times

for ii in range(8):
    # parallel run of simulations -- this generates approximately 1 million simulations per hour
    # this cell takes about an hour to run
    p = Pool(cpu_count()-2)
    stt = time.time()   
    bb = p.map(multiProcTraj.flyBug_firstLast, range(100000) ) 
    print(time.time() -stt)

    p.close()
    p.join()
    print(time.time() -stt)
    newDF = pd.DataFrame(bb, columns= ["x0", "xf", "xd0", "xdf", "y0", "yf", "yd0", "ydf", "theta0", \
            "thetaf", "thetad0", "thetadf", "phi0", "phif", "phid0", "phidf", "F", "alpha", "tau0"])
    
    tstamp = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
    print(tstamp)
    
    newDF.to_csv(os.path.join(pythonMadeData, 
                          "RandomInitCond_" + tstamp + ".csv"),
             index = False)

12.170493364334106
12.538028240203857
2019_04_03__10_12_58
12.431798934936523
12.785820722579956
2019_04_03__10_13_13
12.602301359176636
12.951338768005371
2019_04_03__10_13_28
12.12761116027832
12.53690242767334
2019_04_03__10_13_43
12.473654508590698
12.855144739151001
2019_04_03__10_13_58
12.161490678787231
12.53443956375122
2019_04_03__10_14_13
12.238284587860107
12.629096984863281
2019_04_03__10_14_28
12.2592294216156
12.65784215927124
2019_04_03__10_14_43


In [19]:
# refref: one million simulations per 96 seconds?

In [20]:
# for loop to run parallel processes a few times
for ii in range(8):
    # parallel run of simulations -- this generates approximately 1 million simulations per hour
    p = Pool(cpu_count()-4)
    stt = time.time()   
    bb = p.map(multiProcTraj.flyBug_firstLast, range(100000) ) 
    print(time.time() -stt)

    p.close()
    p.join()
    print(time.time() -stt)
    newDF = pd.DataFrame(bb, columns= ["x0", "xf", "xd0", "xdf", "y0", "yf", "yd0", "ydf", "theta0", \
            "thetaf", "thetad0", "thetadf", "phi0", "phif", "phid0", "phidf", "F", "alpha", "tau0"])
    
    tstamp = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
    print(tstamp)
    
    newDF.to_csv(os.path.join(pythonMadeData, 
                          "RandomInitCond_" + tstamp + ".csv"),
             index = False)

12.143568515777588
12.482109308242798
2019_04_03__10_14_58
12.04582667350769
12.371912240982056
2019_04_03__10_15_13
12.307137966156006
12.702399730682373
2019_04_03__10_15_28
12.314082384109497
12.674564123153687
2019_04_03__10_15_43
12.46969723701477
12.779642105102539
2019_04_03__10_15_58
11.989979028701782
12.280172109603882
2019_04_03__10_16_13
12.210386037826538
12.581607818603516
2019_04_03__10_16_28
12.374919414520264
12.690775394439697
2019_04_03__10_16_43


In [21]:
# for loop to run parallel processes a few times

for ii in range(8):
    # parallel run of simulations -- this generates approximately 1 million simulations per hour
    # this cell takes about an hour to run
    p = Pool(cpu_count()-2)
    stt = time.time()   
    bb = p.map(multiProcTraj.flyBug_firstLast, range(100000) ) 
    print(time.time() -stt)

    p.close()
    p.join()
    print(time.time() -stt)
    newDF = pd.DataFrame(bb, columns= ["x0", "xf", "xd0", "xdf", "y0", "yf", "yd0", "ydf", "theta0", \
            "thetaf", "thetad0", "thetadf", "phi0", "phif", "phid0", "phidf", "F", "alpha", "tau0"])
    
    tstamp = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
    print(tstamp)
    
    newDF.to_csv(os.path.join(pythonMadeData, 
                          "RandomInitCond_" + tstamp + ".csv"),
             index = False)

12.2233247756958
12.632879257202148
2019_04_03__10_16_58
12.080736875534058
12.434906721115112
2019_04_03__10_17_13
12.11264681816101
12.543555736541748
2019_04_03__10_17_28
12.087686777114868
12.438823461532593
2019_04_03__10_17_43
12.020864963531494
12.432202100753784
2019_04_03__10_17_58
12.255271196365356
12.646639585494995
2019_04_03__10_18_13
12.120599031448364
12.56366491317749
2019_04_03__10_18_28
12.12561321258545
12.544893503189087
2019_04_03__10_18_43
